In [1]:
import sklearn
import numpy as np
import os
import h5py
from sklearn.cross_validation import train_test_split

C:\Users\bhask\Anaconda3\envs\tfenv\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
mypath='D://Work//quickdraw//'
name_list=[]
for (dirpath, dirnames, filenames) in os.walk(mypath):
    if filenames != '.DS_Store':
        name_list.extend(filenames)
        break
        
x_train=[]
x_test=[]
y_train=[]
y_test=[]
x_tot=[]
y_tot=[]

slice_train=int(80000/len(name_list))
i=0
seed=np.random.randint(1, 10e6)

for name in name_list:
    path=mypath+name
    x=np.load(path)
    x=x.astype("float32")/255
    y=[i]*len(x)
    np.random.seed(seed)
    np.random.shuffle(x)
    np.random.seed(seed)
    np.random.shuffle(y)
    x=x[:slice_train]
    y=y[:slice_train]
    if i!=0 :
        xtot=np.concatenate((x,xtot), axis=0)
        ytot=np.concatenate((y,ytot), axis=0)
    else:
        xtot=x
        ytot=y
    i+=1
    
x_train, x_test, y_train, y_test=train_test_split(xtot, ytot, test_size=0.2, random_state=42)

In [3]:
import matplotlib.pyplot as plt
xtrain=[]
xtest=[]
for i in x_train:
    xtrain.append(i.reshape(28,28,1))
for i in x_test:
    xtest.append(i.reshape(28,28,1))
xtrain=xtrain[:63900]
y_train=y_train[:63900]

In [4]:
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

def one_hot_encode(x):
    enc=OneHotEncoder(n_values=12)
    ohc_labels=enc.fit_transform(np.array(x).reshape(-1,1)).toarray()
    return ohc_labels

x = tf.placeholder("float", shape=[None,28,28,1], name='x')
y = tf.placeholder("float", shape=[None,12], name='y')
keep_probs = tf.placeholder("float", name='keep_probs')

def conv2d(inp_lyr,ofilters,kernel,stride):
    conv2d_lyr=tf.layers.conv2d(inputs=inp_lyr,
                               filters=ofilters,
                               kernel_size=kernel,
                               strides=stride,
                               padding="same",
                               activation=tf.nn.relu,
                               )
    return conv2d_lyr

def maxpool2d(inp_lyr,pool,stride):
    maxp2d=tf.layers.max_pooling2d(inputs=inp_lyr,
                                 pool_size=pool,
                                 strides=stride,
                                 padding="same")
    return maxp2d

def model():
    img_inp=tf.reshape(x,[-1,28,28,1])
    layer=conv2d(img_inp,32,[7,7],[2,2])
    layer=maxpool2d(layer,[5,5],[2,2])
    
    layer=conv2d(layer,32,[5,5],[1,1])
    layer=conv2d(layer,64,[3,3],[1,1])
    layer=maxpool2d(layer,[3,3],[2,2])
    
    layer=conv2d(layer,64,[3,3],[1,1])
    layer=conv2d(layer,128,[3,3],[1,1])
    layer=conv2d(layer,256,[3,3],[1,1])
    layer=maxpool2d(layer,[3,3],[2,2])
    
    layer=tf.layers.dropout(layer,0.4)
    
    layer=tf.contrib.layers.flatten(layer)
    
    layer=tf.layers.dense(layer,256)
    layer=tf.layers.dense(layer,512)
    layer=tf.layers.dense(layer,1024)
    
    layer=tf.layers.dense(layer,12)
    
    layer=tf.nn.softmax(layer)
    
    return layer

cnn_mod=model()
cross_entropy=-tf.reduce_sum(y*tf.log(cnn_mod))
epoch_batch=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
corr_pred=tf.equal(tf.argmax(cnn_mod, 1), tf.argmax(y, 1))
acc=tf.reduce_mean(tf.cast(corr_pred, tf.float32), name='acc')

X_train=np.split(np.array(xtrain),100)
y_train=np.split(np.array(y_train),100)


def train_model():
    for epoch in range(30):
        for i in range (0,10):
            x_img=X_train[i]
            y_img=one_hot_encode(y_train[i])
            tr_acc=sess.run(acc,feed_dict={x: x_img, y: y_img, keep_probs: 1.0})
            print('Epoch: {} Batch: {} Accuracy = {:.6f}'.format(epoch+1, i+1, tr_acc))
            sess.run(epoch_batch,feed_dict={x: x_img, y: y_img, keep_probs: 0.5})
            
sess=tf.Session()
sess.run(tf.global_variables_initializer())    
train_model()

Epoch: 1 Batch: 1 Accuracy = 0.106416
Epoch: 1 Batch: 2 Accuracy = 0.137715
Epoch: 1 Batch: 3 Accuracy = 0.115806
Epoch: 1 Batch: 4 Accuracy = 0.111111
Epoch: 1 Batch: 5 Accuracy = 0.076682
Epoch: 1 Batch: 6 Accuracy = 0.145540
Epoch: 1 Batch: 7 Accuracy = 0.134585
Epoch: 1 Batch: 8 Accuracy = 0.139280
Epoch: 1 Batch: 9 Accuracy = 0.162754
Epoch: 1 Batch: 10 Accuracy = 0.161189
Epoch: 2 Batch: 1 Accuracy = 0.115806
Epoch: 2 Batch: 2 Accuracy = 0.107981
Epoch: 2 Batch: 3 Accuracy = 0.111111
Epoch: 2 Batch: 4 Accuracy = 0.161189
Epoch: 2 Batch: 5 Accuracy = 0.173709
Epoch: 2 Batch: 6 Accuracy = 0.251956
Epoch: 2 Batch: 7 Accuracy = 0.331768
Epoch: 2 Batch: 8 Accuracy = 0.344288
Epoch: 2 Batch: 9 Accuracy = 0.366197
Epoch: 2 Batch: 10 Accuracy = 0.394366
Epoch: 3 Batch: 1 Accuracy = 0.452269
Epoch: 3 Batch: 2 Accuracy = 0.438185
Epoch: 3 Batch: 3 Accuracy = 0.449139
Epoch: 3 Batch: 4 Accuracy = 0.417840
Epoch: 3 Batch: 5 Accuracy = 0.430360
Epoch: 3 Batch: 6 Accuracy = 0.446009
Epoch: 3 B

Epoch: 22 Batch: 3 Accuracy = 0.820031
Epoch: 22 Batch: 4 Accuracy = 0.843506
Epoch: 22 Batch: 5 Accuracy = 0.799687
Epoch: 22 Batch: 6 Accuracy = 0.823161
Epoch: 22 Batch: 7 Accuracy = 0.834116
Epoch: 22 Batch: 8 Accuracy = 0.818466
Epoch: 22 Batch: 9 Accuracy = 0.838811
Epoch: 22 Batch: 10 Accuracy = 0.823161
Epoch: 23 Batch: 1 Accuracy = 0.827856
Epoch: 23 Batch: 2 Accuracy = 0.820031
Epoch: 23 Batch: 3 Accuracy = 0.818466
Epoch: 23 Batch: 4 Accuracy = 0.849765
Epoch: 23 Batch: 5 Accuracy = 0.801252
Epoch: 23 Batch: 6 Accuracy = 0.830986
Epoch: 23 Batch: 7 Accuracy = 0.838811
Epoch: 23 Batch: 8 Accuracy = 0.816901
Epoch: 23 Batch: 9 Accuracy = 0.849765
Epoch: 23 Batch: 10 Accuracy = 0.824726
Epoch: 24 Batch: 1 Accuracy = 0.832551
Epoch: 24 Batch: 2 Accuracy = 0.829421
Epoch: 24 Batch: 3 Accuracy = 0.810642
Epoch: 24 Batch: 4 Accuracy = 0.848200
Epoch: 24 Batch: 5 Accuracy = 0.815336
Epoch: 24 Batch: 6 Accuracy = 0.821596
Epoch: 24 Batch: 7 Accuracy = 0.815336
Epoch: 24 Batch: 8 Accu

In [8]:
save_model_path='D:/Work/quickdraw'
saver = tf.train.Saver()
save_path = saver.save(sess, save_model_path)

In [9]:
def test_model():
    
    loaded_graph = tf.Graph()

    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        # Get Tensors from loaded model
        loaded_x = loaded_graph.get_tensor_by_name('x:0')
        loaded_y = loaded_graph.get_tensor_by_name('y:0')
        loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_probs:0')
        loaded_acc = loaded_graph.get_tensor_by_name('acc:0')
        
        # Get accuracy in batches for memory limitations
        test_batch_acc_total = 0
        test_batch_count = 0
        
        for i in range(0,2):
            x_test_img=X_test[i]
            y_test_img=one_hot_encode(y_test[i])
            test_batch_acc_total += sess.run(
                loaded_acc,
                feed_dict={loaded_x: x_test_img, loaded_y: y_test_img, loaded_keep_prob: 1.0})
            test_batch_count += 1

        print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))

xtest=xtest[:12000]
y_test=y_test[:12000]
X_test=np.split(np.array(xtest),2)
y_test=np.split(np.array(y_test),2)
test_model()

INFO:tensorflow:Restoring parameters from D:/Work/quickdraw
Testing Accuracy: 0.8154166638851166

